# Event Driven Programming


The goal of this project is to build a _framework_ that allows us to design interactive applications 
such as animations and games using our own event driven programmning system.  We will also program a classic arcade game in this system according to the specification. Once again, we will describe the framework here and what you need to do. Your goal is to implement the missing functions.

The overall system has three parts: 
- MainEventLoop : this maintains a list of _entities_ and list of _events_.
- Entities are objects that implement the `handleEvent` method in order to handle an event e. As a result, the entity gets transformed into a new list of entities, and generates a new list of events. 
- Events are broadcast to all entities. Each event implements an `appliesTo` method that defines all entities that it can be broadcast to.

#### Entities 
Entities are described in the file `Entity.scala`. The main contents are reproduced below.
- An entity has a unique integer `id`.
- An entity __must__ implement a function called `handleEvent` that implements how an event `e` transforms an entity. As a result, the entity can be transformed into a list of zero, one or multiple entities. Also, new events can be generated in this process.
- An entity may optionally implement a render function with a `Graphics2D` handle, or if it does not the entity is not drawn on the canvas.

In [ ]:
trait Entity {
    // An entity must have a unique integer ID.
    val id: Int 
    // An entity must implement a function handleEvent that takes in a handle of a global MainEventLoop and an event.
    // It must return a list of transformed entities, and a list of freshly generated events.
    def handleEvent(global: MainEventLoop) (e: Event): (List[Entity], List[Event]) 
    // Since we are supporting animations, an entity may optionally implement a function to draw itself on the canvas
    // We are using Swing library here -- you can safely ignore these functions unless you are interested in how Swing works.
    def render(g: Graphics2D): Unit = ()
}

#### Events 
Entities are closely tied to Events. Events are defined in the file `Event.scala`. The main definition of an event is as follows.  Note that an event must define a function called `appliesTo` that takes in an entity `e` and returns a Boolean `true` if the event can be sent to the entity `e` and `false` otherwise

In [ ]:
trait Event {
    // An event must define a function called appliesTo
    //   that takes in an entity e and returns a Boolean 
    //   true if the event can be sent to the entity e and false otherwise
    def appliesTo(e: Entity): Boolean = true 
}

We define a few inbuilt events already.
- `Tick(delta)`: This is a clock tick event that is generated for a fixed time period delta. The programmer has a way of setting this delta in our main event loop.
- `Stop`: This is a special event that must be handled by all entities. Whenever an entity gets a `Stop` event, it must return the __empty list__ of entities and the __empty list__ of events. As we will see the overall system will be set up so that Stop kills all entities and stops the trampoline.
- `MouseClick(x,y)`: These are registering a mouse click event at coordinates `(x,y)` on the canvas.


In [ ]:
// From Event.scala file -- we have defined three types of Events that are common for most applications.
case class Tick(delta: Double) extends Event
case class MouseClick(x: Double, y: Double) extends Event
case object Stop extends Event

#### Main Event Loop

The main event loop is the core of the system. It is what you are going to be asked to develop. It contains a trampoline or a central event loop and some utility functions. We will explain the detailed function of the main event loop later. For the time being it suffices to understand the following facts:
- The main event loop maintains a list of current entities.
- It maintains a list of current _unprocessed_ events.
- It periodically takes each unprocessed event and delivers it to all the entities to which the event's `appliesTo` function returns true. 
  - It calls the corresponding `handleEvent` function of the entity with the corresponding event.
  - It replaces the entity by the list of entities returned by the handleEvent. 
  - It adds the newly generated events back to the event queue.
  
 <img src="entity-event-handling.png" width="25%">
 
__Important__ When an entity handles an event, it is implicitly _deleted_ and transformed into a set of new entities. The figure shows an `OldEntity` handling an event and transforming itself into three new entities. However, it may be the case that an entity changes into $0$ new entities: which is how an entity is stopped.
An entity may change into one new entity. Or sometimes the new entity may simply be the old entity unchanged. All of these patterns are going to be useful in our event driven programming framework.

## Main Event Loop 

<font color="red"> Note the following important rules to encourage you to program functionally and use what you have learned in this class. </font>
- The use of for/while loops and other mutable vars other than the two fields `entities` and `events` is forbidden.
- We will scan your code for usage of `for/while/do-until loops` and identifiers declared as `var` : if we find them, the solution is invalid.
- Valid solutions should use functors such as `map`, `foldLeft/foldRight`, `filter` and `partition`.


## Task: Program a classic arcade game in our DSL

In this task you are asked to program a classic arcade game using the framework created thus far.

The arcade game has baloons that float around in the screen and a dart gun that shoots darts at the baloon.

<img src="screenshot-arcade-game.png" width="20%" height="20%">

The goal is to create appropriate set of entities and events according to the specifications below.

### Entities

The entities for this game include 
- `ArcadeGameManager(gamePoints: Int)` an entity that keeps track of game points. It is responsible for spawning new baloons at random intervals.
- `ArcadeBaloon(id: Int, x: Double, y: Double, driftX: Double)` represents a baloon on the screen with coordinates `(x,y)` and speed `driftX` along X direction. Assume that the baloon does not move vertically.
- `Dart(id: Int, x: Double, y: Double)` a dart that is currently flying up with position `(x,y)`
- `DartShooter(x: Int, numDarts: Int)` a dart shooter that is currently at position x : the y position is always fixed. Further it has `numDarts` darts remaining to shoot.

### Events

The relevant events are 
- `Tick(delta)`: delta seconds have elapsed since the last tick.
  -  This event applies to all entities.
- `Stop`:  Stop the game by taking away all the entities.
  - This event applies to all entities.
- `Move(dx:Int)`: Move the dart shooter's x position by dx.
  - This event must apply only to the `DartShooter` entity.
- `ShootDart`: Shoot a dart from the dart shooter.
  - This event must apply only to the `DartShooter` entity.
- `BaloonEscaped`: A baloon escaped without getting shot
  - This event must apply only to the `ArcadeGameManager` entity.
- `BaloonBurst`: A baloon was burst by colliding with a dart.
  - This event applies to the `DartShooter` and `ArcadeGameManager` entities.
- `Boom(x,y)`: signals the current position of a dart and used to detect collision between a dart and a baloon.
  - This event applies only to the `ArcadeBaloon(id, x1,y1, drift) ` entity if and only if the distance between the dart and the baloon: $\sqrt{(\texttt{x-x1})^2 + (\texttt{y-y1})^2} \leq \texttt{ScreenDimensions.baloonBurstDistance}$. 

We will now specify how each entity behaves on the events it must handle.

__Task 2.1__ Define the `appliesTo` method for each of the relevant events in the file `ArcadeGame.scala`.

## Entity applications

We will not provide information that will help you implement/complete the `appliesTo` function for each of the events. You will see the code has 'TODO's near the top of the ArcadeGame.scala with comments matching this
- Move: Define the appliesTo method so that this event can only be sent to the DartShooter Entity
- ShootDart: Define the appliesTo method so that this event can only be sent to the DartShooter Entity
- BaloonEscaped: Define the appliesTo method so that this event can only be sent to the ArcadeGameManager Entity
- BaloonBurst: Define the appliesTo method so that this can only be sent to the ArcadeGameManager Entity
- Boom(_,_): Boom appllies to an arcade baloon iff euclidiean distance between (x1,y1) and (x,y) is less than or equal to ScreenDimensions.baloonBurstDistance

## Event Handler Specification

We will now provide information that will help you implement/complete the `handleEvent` function for each of the entities. You will see that parts of this are already implemented for you

### ArcadeGameManager

- NOTE: ArcadeGameManager is the most complicated to code. You may want to do this one last.
- Event: `Tick(delta)`: this event will randomly create a new baloon while leaving the ArcadeGameManager unaltered.
  - Use the Random number generator in scala to generate a random double precision between 0.0 and 1.0. The function `Random.nextDouble` should help.
  - If the random number is less than `0.02` then you must return a list of two entities: the `ArcadeGameManager` itself unchanged, and a  new `ArcadeBaloon`:
    - The baloon has its `id` given by `global.numEntities+1`, choose the x position to be 20, y position randomly between `500 and 600`, and driftX to be a random number between `4` and `12`. Such a baloon with appear to move from left to right on the screen.
  - If the random number is greater than `0.98` then you must return a list of two entities: the `ArcadeGameManager` itself unchanged,  and a new `ArcadeBaloon`:
    - The baloon has its `id` given by `global.numEntities+1`, choose the x position to be `ScreenDimensions.width-20`, y position randomly between `500 and 600`, and driftX to be a random number between `-12` and `-4`. This baloon will appear to move from right to left on the screen.
  - Else, just return the `ArcadeGameManager` unchanged.
  - In all cases, no new events are generated.
- Event: `BaloonEscaped`
  - Returns a list of entities consisting of a single `ArcadeGameManager` with same id but with the `gamePoints + ScreenDimensions.pointsBaloonEscaped` as the new value of `gamePoints`.
  - No new events are generated.
- Event: `BaloonBurst`
  - Returns a list of entities consisting of a single `ArcadeGameManager` with same id but with the `gamePoints + ScreenDimensions.pointsBaloonBurst` as the new value of `gamePoints`.
  - No new events are generated.
- Event: `Stop`
  - Return empty list of entities and empty list of events.
- All other events
  - Return the same ArcadeGameManager unchanged and no new events.
  
### ArcadeBaloon

- Event: `Tick(delta)`: this event will cause the baloon to move according to its current drift. If the baloon tries to leave the bounds of the screen in this process, the entity is not renewed and a BaloonEscaped event is generated.
  - Compute `x1 = x + delta * driftX`. The y position remains unchanged (you can make a random change to it to simulate the effect of a baloon bobbing up and down randomly, if you wish).
  - If (x1 >= ScreenDimensions.shooterMax or x1 <= 0) then return the empty list of entities and the list of events consiting of the single event `BaloonEscaped`.
  - Else, return a list of entities consiting of the single entity `ArcadeBaloon(id, x1, y, driftX)` and the empty list of events.
  
- Event: `Boom(_,_)`
  - Return the empty list of entities and the list consisting of a single `BaloonBurst` event.
- Event: `Stop`
  - Return empty list of entities and empty list of events.
- All other events
  - Return the same ArcadeBaloon unchanged and no new events.
  
### Dart
- Event: `Tick(delta)`: the dart will move up the screen using velocity given by `ScreenDimensions.dartSpeed`.
  - Compute `y1 = y - delta * ScreenDimensions.dartSpeed`.
  - If `y1 <= 20` then return the empty list of entities and empty list of events.
  - Else, return the list consisting of `Dart(id, x, y1)` and a list consisting of the single event `Boom(x,y1)`.
- Event: `Stop`
  - Return empty list of entities and empty list of events.
- All other events
  - Return the same `Dart` unchanged and no new events.

### DartShooter
- Event: `Move(dx)`
  - Compute `x1 = x + dx`
  - If (x1 <= `ScreenDimensions.shooterMin` or x1 >= `ScreenDimensions.shooterMax`) then 
    - return a list of single entity that consist of the shooter unchanged and no events.
  - Else 
    - return a list of single entity `DartShooter(x1, numDarts)` and no events.
- Event: `ShootDart`
  - If (`numDarts > 0`)
    - Create a new `Dart` with `id = global.numEntities+1`, x = `x`, y = `y - 70`.
    - Return a list of entities consisting of `DartShooter(x, numDarts-1)` and the new Dart.
    - The empty list of events are created.
  - Else,
    - Return a list of entities consisting of just one entity : the `DartShooter` object itself unchanged.
    - The empty list of events are created.
- Event: `BaloonBurst`: The number of darts must increase by `ScreenDimensions.numDartsAdded`
- Event: `Stop`
  - Return empty list of entities and empty list of events.
- All other events
  - Return the same DartShooter unchanged and no new events.


Once you have completed handling of all these events. Your game is ready to play.
We have written unit tests under ArcadeGameTests that your code should pass.